In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('daily_minimum_temps.csv', parse_dates = ["Date"],index_col= "Date") 

C:\Users\achyu\AppData\Local\Temp\ipykernel_11412\4272664007.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('daily_minimum_temps.csv', parse_dates = ["Date"],index_col= "Date")


In [3]:
df.head()

,Temp
Date,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8


In [4]:

df.isna().sum(axis = 0)

Temp    0
dtype: int64

In [5]:
df.shape

(3650, 1)

In [6]:
df["Temp"] = pd.to_numeric(df["Temp"], errors = 'coerce')
df = df.dropna()

In [7]:
df.isna().sum(axis = 0)

Temp    0
dtype: int64

In [8]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df['Temp'].values.reshape(-1,1))

In [9]:
seq_length = 30

In [10]:
def create_sequences(data_scaled, seq_length):
    X, y = [], []
    for i in range(len(data_scaled) - seq_length):
        X.append(data_scaled[i:i + seq_length])
        y.append(data_scaled[i + seq_length])
    return np.array(X), np.array(y)


In [11]:
X, y = create_sequences(data_scaled, seq_length)


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

In [13]:
#building thr RNN model
model = Sequential([
    LSTM(64, activation='relu', input_shape=(seq_length, 1)),
    Dense(1)
])


C:\Users\achyu\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
model.compile(optimizer='adam', loss='mse')

In [15]:
model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0729
Epoch 2/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0116
Epoch 3/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0117
Epoch 4/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0103
Epoch 5/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0111
Epoch 6/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0114
Epoch 7/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0108
Epoch 8/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0101
Epoch 9/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0109
Epoch 10/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0101
Epoch 11/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0100
Epoch 12/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0101
Epoch 13/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0094
Epoch 14/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0095
Epoch 15/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0091
Epoch 16/20

In [16]:
print(np.isnan(data_scaled).sum())

0


In [17]:
y_pred_scaled = model.predict(X_test)
y_pred_scaled = np.clip(y_pred_scaled, 0, 1)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_actual = scaler.inverse_transform(y_test)

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [18]:
last_sequence = data_scaled[-seq_length:].reshape(1, seq_length, 1)
next_temp_scaled = model.predict(last_sequence)
next_temp_scaled = np.clip(next_temp_scaled, 0, 1)
next_day_temp = scaler.inverse_transform(next_temp_scaled)

print(f"Next day's temperature is : ", next_day_temp[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Next day's temperature is :  13.406241


In [19]:
import joblib
joblib.dump(model, 'temperature_prediction_model.pkl')
joblib.dump(scaler, 'temperature_scaler.pkl')


['temperature_scaler.pkl']